In [56]:
import pyquery
import re

In [66]:
import pyquery
import requests
import re
import json
import pandas as pd
import win32clipboard

trans_url = "https://warframe.huijiwiki.com/wiki/UserDict"
print("获取翻译。。。")
trans_content = pyquery.PyQuery(trans_url)("#mw-content-text").html()
trans_items = re.findall('<tr><th>([^<>]+?)</th><td class="value">"([^<>]+?)"</td></tr>', trans_content.replace("&amp;", "&"))
trans_dict = {k:v for v, k in trans_items}
retrans_dict = {k:v for k, v in trans_items}

ua = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36"

def get_wm_data(url):
    print("Get From:", url)
    res = requests.get(url, headers={"User-Agent": ua})
    pq = pyquery.PyQuery(res.text)
    script = pq("script#application-state").html()
    return json.loads(script)

print("获取物品列表。。。")
wm_url = "https://warframe.market/"
wm_info = get_wm_data(wm_url)
wm_items = {item['item_name']:item['url_name'] for item in wm_info['items']['en']}


add_ons = [
    ("Primed ", ""),
    
    ("", ""),
    ("", " Set"),
    ("", " Prime"),
    ("", " Prime Set"),
    ("", " Vandal"),
    ("", " Vandal Set"),
    ("", " Wraith"),
    ("", " Wraith Set"),
    ("Wraith ", " Set"),
    ("Prisma ", ""),
    ("", " Ayatan Sculpture"),
    
    ("Rakta ", ""),
    ("Sancti ", ""),
    ("Secura ", ""),
    ("Synoid ", ""),
    ("Telos ", ""),
]

def en2ch(name):
    if name in retrans_dict:
        return retrans_dict[name]
    for en, ch in trans_items:
        name = name.replace(en, ch)
    return name

获取翻译。。。
获取物品列表。。。
Get From: https://warframe.market/


In [62]:
group_names = "钢铁防线 • 均衡仲裁者 • 中枢苏达 • 佩兰数列 • 血色面纱 • 新世间".split(" • ")
group_names

['钢铁防线', '均衡仲裁者', '中枢苏达', '佩兰数列', '血色面纱', '新世间']

In [65]:
items = {}
groups = {}
for group_name in group_names:
    print(group_name)
    groups[group_name] = {}
    pq = pyquery.PyQuery("https://warframe.huijiwiki.com/wiki/%s" % group_name)
    tmp = pq("div.mw-parser-output")
    tmp("h2:contains(声望奖励)").prevAll().remove()
    tmp("h2:contains(语录)").nextAll().remove()
    for item in tmp("div.mw-parser-output > div > div").items():
        item = pyquery.PyQuery(item)
        num = ''.join(re.findall("\d", item("div > span").text()))
        num = int(num or "0")
        name = item("a > span").text()
        if num:
            print(num, name)
            if name not in items:
                items[name] = {"cost": num, "from": [group_name]}
            else:
                items[name]["from"].append(group_name)
            groups[group_name][name] = num

钢铁防线
2500 5x Roller 魅影
2500 反抗纹章
5000 舰队纹章
7500 警戒纹章
20000 Syndicate Relic Pack (3 X Random Void Relics)
20000 Kaszas Handle
200000 Velocitus Receiver
20000 Corvas Stock
10000 起义纹章
15000 保卫关系纹章
25000 大型团队医疗回复
12000 自由战士纹章
17000 装甲纹章
20000 Agkuza Guard
20000 Fluctus Stock
50000 Insignia Sculpture
25000 散射正义(Hek)
25000 正义刀锋(Dual Cleavers)
25000 破碎正义(Sobek)
15000 反叛纹章
20000 顽抗纹章
25000 化像之道(Atlas)
25000 构造裂缝(Atlas)
25000 矿石凝视(Atlas)
25000 巨大石者 (Atlas)
25000 狂热火球 (Ember)
25000 闪耀助燃 (Ember)
25000 惊惧热浪(Ember)
25000 烈焰爆震 (Ember)
25000 涌流突进(Excalibur)
25000 终结闪光(Excalibur)
25000 狂怒标枪(Excalibur)
25000 华彩刀剑 (Excalibur)
25000 寒冰之力 (Frost)
25000 滞痕冰浪 (Frost)
25000 冰封护罩 (Frost)
25000 冰冷雪崩 (Frost)
25000 弹道靶心 (Mesa)
25000 失衡护盾 (Mesa)
25000 枪口闪焰(Mesa)
25000 火成碎流 (Nezha)
25000 火绫守护 (Nezha)
25000 中子星爆(Nova)
25000 反物质吸收 (Nova)
25000 脱离速度 (Nova)
25000 惩击洗礼 (Oberon)
25000 圣域爆发 (Oberon)
25000 凤凰新生 (Oberon)
25000 神圣清算 (Oberon)
25000 铁甲冲锋 (Rhino)
25000 碎铁弹片(Rhino)
25000 刺骨战吼(Rhino)
25000 践踏加固 (Rhino)
25000 猛毒附

In [74]:
items = {}
for group in groups:
    print(group)
    for name, cost in groups[group].items():
        if "(" in name:
            idx = name.find("(")
            name = en2ch(name[:idx].strip()) + name[idx:].strip()
        else:
            name = en2ch(name.strip())
        print(name, cost)
        if name not in items:
            items[name] = {"cost": num, "from": [group]}
        else:
            items[name]["from"].append(group)

钢铁防线
5x 滚子 魅影 2500
反抗纹章 2500
舰队纹章 5000
警戒纹章 7500
集团 遗物 Pack(3 X Random Void Relics) 20000
死亡使徒 握柄 20000
极速电磁步枪 枪机 200000
黑鸦 枪托 20000
起义纹章 10000
保卫关系纹章 15000
大型团队医疗回复 25000
自由战士纹章 12000
装甲纹章 17000
驯象钩刃 护手 20000
巨浪 枪托 20000
佩章雕塑 50000
散射正义(Hek) 25000
正义刀锋(Dual Cleavers) 25000
破碎正义(Sobek) 25000
反叛纹章 15000
顽抗纹章 20000
化像之道(Atlas) 25000
构造裂缝(Atlas) 25000
矿石凝视(Atlas) 25000
巨大石者(Atlas) 25000
狂热火球(Ember) 25000
闪耀助燃(Ember) 25000
惊惧热浪(Ember) 25000
烈焰爆震(Ember) 25000
涌流突进(Excalibur) 25000
终结闪光(Excalibur) 25000
狂怒标枪(Excalibur) 25000
华彩刀剑(Excalibur) 25000
寒冰之力(Frost) 25000
滞痕冰浪(Frost) 25000
冰封护罩(Frost) 25000
冰冷雪崩(Frost) 25000
弹道靶心(Mesa) 25000
失衡护盾(Mesa) 25000
枪口闪焰(Mesa) 25000
火成碎流(Nezha) 25000
火绫守护(Nezha) 25000
中子星爆(Nova) 25000
反物质吸收(Nova) 25000
脱离速度(Nova) 25000
惩击洗礼(Oberon) 25000
圣域爆发(Oberon) 25000
凤凰新生(Oberon) 25000
神圣清算(Oberon) 25000
铁甲冲锋(Rhino) 25000
碎铁弹片(Rhino) 25000
刺骨战吼(Rhino) 25000
践踏加固(Rhino) 25000
猛毒附加(Saryn) 25000
蜕化再生(Saryn) 25000
感染毒雾(Saryn) 25000
斗士纹章 30000
勇气玛瑞火枪 100000
勇气海克 125000
勇气恶

In [107]:
weapons = """什长双枪 • 凯旋将军 • 合成燃气炮 • 巨浪 • 极速电磁步枪 • 格拉特勒 • 菲德菈 • 菲德拉 • 黑鸦 • 克纳克斯 • 半人马 • 奥努里克斯 • 拉斯波恩 • 死亡使徒 • 真理巨剑 • 驯象钩刃""".split(" • ")
already_get = tuple("""
极速电磁步枪
半人马
奥努里克斯
合成燃气炮 枪托
菲德拉 枪托
菲德拉 枪机
""".strip().splitlines())
weapons = tuple(weapons)

In [109]:
for group in groups:
    print(group)
    for name, cost in groups[group].items():
        if "(" in name:
            idx = name.find("(")
            name = en2ch(name[:idx].strip()) + name[idx:].strip()
        else:
            name = en2ch(name.strip())
        if name.startswith(weapons) and not name.startswith(already_get):
            print(name)
    print()

钢铁防线
死亡使徒 握柄
极速电磁步枪 枪机
黑鸦 枪托
驯象钩刃 护手
巨浪 枪托

均衡仲裁者
什长双枪 枪管
菲德菈 枪管
黑鸦 枪管
合成燃气炮 枪管
半人马 Aegis

中枢苏达
什长双枪 枪机
极速电磁步枪 枪管
黑鸦 枪机
合成燃气炮 枪机
巨浪 枪管

佩兰数列
奥努里克斯 握柄
菲德拉 枪机
半人马 剑刃
合成燃气炮 枪托

血色面纱
死亡使徒 刀刃
极速电磁步枪 枪托
拉斯波恩 握柄
驯象钩刃 握柄
巨浪 弓臂

新世间
奥努里克斯 刀刃
菲德拉 枪托
拉斯波恩 锤头
驯象钩刃 刀刃
半人马 握柄



In [110]:
keys = list(items.keys())
keys.sort()

for item in keys:
    if item.startswith(weapons):
        print(item, items[item]["from"])

什长双枪 枪机 ['中枢苏达']
什长双枪 枪管 ['均衡仲裁者']
半人马 Aegis ['均衡仲裁者']
半人马 剑刃 ['佩兰数列']
半人马 握柄 ['新世间']
合成燃气炮 枪托 ['佩兰数列']
合成燃气炮 枪机 ['中枢苏达']
合成燃气炮 枪管 ['均衡仲裁者']
奥努里克斯 刀刃 ['新世间']
奥努里克斯 握柄 ['佩兰数列']
巨浪 弓臂 ['血色面纱']
巨浪 枪托 ['钢铁防线']
巨浪 枪管 ['中枢苏达']
拉斯波恩 握柄 ['血色面纱']
拉斯波恩 锤头 ['新世间']
极速电磁步枪 枪托 ['血色面纱']
极速电磁步枪 枪机 ['钢铁防线']
极速电磁步枪 枪管 ['中枢苏达']
死亡使徒 刀刃 ['血色面纱']
死亡使徒 握柄 ['钢铁防线']
菲德拉 枪托 ['新世间']
菲德拉 枪机 ['佩兰数列']
菲德菈 枪管 ['均衡仲裁者']
驯象钩刃 刀刃 ['新世间']
驯象钩刃 护手 ['钢铁防线']
驯象钩刃 握柄 ['血色面纱']
黑鸦 枪托 ['钢铁防线']
黑鸦 枪机 ['中枢苏达']
黑鸦 枪管 ['均衡仲裁者']


In [38]:
dir(tmp)

['Fn',
 '__add__',
 '__call__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__html__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__unicode__',
 '__weakref__',
 '_append',
 '_base_url',
 '_copy',
 '_css_to_xpath',
 '_extend',
 '_filter_only',
 '_get_root',
 '_next_all',
 '_parent',
 '_prev_all',
 '_translator',
 '_translator_class',
 '_traverse',
 '_traverse_parent_topdown',
 'addClass',
 'add_class',
 'after',
 'append',
 'appendTo',
 'append_to',
 'attr',
 'base_url',
 'before',
 'children',
 'clear',
 'clone',
 'closest',
 'contents',
 'copy',
 'count',
 'css',


In [7]:
text="""VoidProjectionsSilverD.png 中纪 B2 遗物 11.11%	50 Endo64.png 内融核心 6.25%	80 Endo64.png 内融核心 2.26%
VoidProjectionsSilverD.png 中纪 S7 遗物 11.11%	VoidProjectionsGoldD.png 后纪 V7 遗物 6.25%	VoidProjectionsGoldD.png 后纪 A3 遗物 9.68%
VoidProjectionsSilverD.png 中纪 M1 遗物 11.11%	VoidProjectionsGoldD.png 后纪 K2 遗物 6.25%	VoidProjectionsGoldD.png 后纪 V7 遗物 9.68%
VoidProjectionsSilverD.png 中纪 Z1 遗物 11.11%	VoidProjectionsGoldD.png 后纪 O2 遗物 6.25%	VoidProjectionsGoldD.png 后纪 H3 遗物 9.68%
VoidProjectionsSilverD.png 中纪 B4 遗物 11.11%	VoidProjectionsGoldD.png 后纪 V6 遗物 6.25%	VoidProjectionsGoldD.png 后纪 N5 遗物 9.68%
VoidProjectionsSilverD.png 中纪 K1 遗物 11.11%	VoidProjectionsGoldD.png 后纪 H3 遗物 6.25%	VoidProjectionsGoldD.png 后纪 V6 遗物 9.68%
FusionCorePackSilversingle.png 戏法增幅 11.11%	VoidProjectionsGoldD.png 后纪 A3 遗物 6.25%	VoidProjectionsGoldD.png 后纪 E2 遗物 9.68%
FusionCorePackSilversingle.png 弹匣增幅 11.11%	VoidProjectionsGoldD.png 后纪 E2 遗物 6.25%	VoidProjectionsGoldD.png 后纪 K2 遗物 9.68%
FusionCorePackSilversingle.png 生命力 11.11%	VoidProjectionsGoldD.png 后纪 N5 遗物 6.25%	VoidProjectionsGoldD.png 后纪 O2 遗物 9.68%
FusionCorePackSilversingle.png 电击触点 6.25%	FusionCorePackSilversingle.png 盗贼大师 2.26%
FusionCorePackSilversingle.png 黄蜂蛰刺 6.25%	FusionCorePackSilversingle.png 分裂膛室 2.26%
FusionCorePackSilversingle.png 简化 6.25%	FusionCorePackSilversingle.png 破灭 2.26%
FusionCorePackSilversingle.png 地狱弹膛 6.25%	FusionCorePackSilversingle.png 合金钻头 2.26%
FusionCorePackSilversingle.png 深层冷冻 6.25%	FusionCorePackSilversingle.png 持久力 2.26%
FusionCorePackSilversingle.png 北风 6.25%	FusionCorePackSilversingle.png 延伸 2.26%
FusionCorePackSilversingle.png 增幅线圈 6.25%	FusionCorePackSilversingle.png 痉挛 2.26%
FusionCorePackSilversingle.png 翻筋斗 2.26%
FusionCorePackSilversingle.png 弱点感应 2.26%"""

In [8]:
import re

In [9]:
items = re.findall("后纪 .. 遗物 ([.\d]+)%", text)

In [10]:
sum([float(i) for i in items]) / 100

1.2744000000000002

In [12]:
1.2744/4.5

0.2832

In [13]:
2/3.5

0.5714285714285714

In [14]:
0.7744/1.5

0.5162666666666667